# REM API Demo With `requests`

This notebook demonstrates how we can call the REM API to get an estimate
for the savings we can expect if we make an electrical upgrade to a home
at a particular address.

## Imports and Configuration

In [2]:
import requests
from pathlib import Path
import pandas as pd

In [3]:
HOST = "https://api.rewiringamerica.org"
REM_ADDRESS_URL = f"{HOST}/api/v1/rem/address"

API_KEY = None  # Put your API key here, or better yet in the file ~/.rwapi/api_key.txt

In [4]:
if API_KEY is None:
    api_key_path = Path.home() / ".rwapi" / "api_key.txt"

    if api_key_path.is_file():
        with open(api_key_path) as f:
            API_KEY = f.read().strip()

## Parameters

Address we are interested in and the upgrade we want to do.

In [5]:
address = "8009 Belmont Ave., Lubbock, TX 79424"
upgrade = "hvac__heat_pump_seer24_hspf13"
heating_fuel = "natural_gas"

## Make the Request

In [6]:
headers = {"Authorization": f"Bearer {API_KEY}"}

response = requests.get(
    url=REM_ADDRESS_URL,
    headers=headers,
    params=dict(address=address, upgrade=upgrade, heating_fuel=heating_fuel),
)

## Pull out the results

We are specifically interested in the total dollar savings.

In [7]:
data = response.json()

In [8]:
annual_savings = -data["fuel_results"]["total"]["delta"]["cost"]["mean"]["value"]

f"Expected annual savings: ${annual_savings:.2f}"

'Expected annual savings: $681.78'

## Modeling Only Baseline in REM

REM typically provides three estimates for a home: one without an upgrade, one with the requested upgrade, and the difference (delta) between them.
For a detailed explanation of these statistics, refer to the [All About REM Statistics notebook](https://github.com/rewiringamerica/api_demos/blob/main/notebooks/All%20About%20REM%20Statistics.ipynb).

A special case occurs when `baseline` is requested as the upgrade. In this case, REM returns a subset of the expected data structure.
For each `fuel_type` within `fuel_results`, the `upgrade` and `delta` fields will not be populated. 
Requesting `baseline` tells REM to perform estimation without applying any upgrades.

Performing a request with only `baseline` can be done similarly to other requests.

In [9]:
headers = {"Authorization": f"Bearer {API_KEY}"}

response = requests.get(
    url=REM_ADDRESS_URL,
    headers=headers,
    params=dict(address=address, upgrade="baseline", heating_fuel=heating_fuel),
)

### Pull out the results for a baseline request

We will extract the `fuel_results`, as we have done previously. In the response, `baseline` contains valid values, while `upgrade` and `delta` are null, as expected since no upgrade was applied to the home.

In [10]:
data = response.json()
fuel_results = data["fuel_results"]

In [11]:
# Convert the baseline from JSON to a pandas data frame for further
# analysis.
baseline = fuel_results[heating_fuel]["baseline"]


def stats_df(baseline):
    return pd.DataFrame(
        [
            {"metric": metric, "stat": stat} | value
            for metric, stats in baseline.items()
            for stat, value in stats.items()
        ]
    )


stats_df(baseline)

,metric,stat,value,unit
0,energy,mean,803.1256,therm
1,energy,median,770.1843,therm
2,energy,percentile_20,562.2991,therm
3,energy,percentile_80,993.4048,therm
4,emissions,mean,5364.7319,kgCO2e
5,emissions,median,5144.6899,kgCO2e
6,emissions,percentile_20,3756.0548,kgCO2e
7,emissions,percentile_80,6635.7620,kgCO2e
8,cost,mean,1278.8299,$
9,cost,median,1237.6351,$


In [12]:
fuel_results[heating_fuel]["upgrade"]

In [13]:
fuel_results[heating_fuel]["delta"]

## Modeling Water Heater Upgrades

The `heating_fuel` of a home is always required by `/api/v1/rem/address`. When you request a water heater upgrade, you must also provide the fuel type of the existing water heater in the `water_heater_fuel` parameter.

In [14]:
# Model a heat pump water heater upgrade

response = requests.get(
    url=REM_ADDRESS_URL,
    headers=headers,
    params=dict(address=address, upgrade="water_heater__heat_pump_uef3.35", heating_fuel=heating_fuel, water_heater_fuel="electricity"),
)
data = response.json()
annual_savings = -data["fuel_results"]["total"]["delta"]["cost"]["mean"]["value"]

f"Expected annual savings: ${annual_savings:.2f}"

'Expected annual savings: $206.80'

You can optionally set `water_heater_fuel` for other upgrades as well. This can help make the results more accurate, by restricting the modeled homes to those with water heaters fueled in the same way.

However, if your water heater fuel type is relatively unusual for homes in your area with your heating fuel type, there is a chance the model won't be able to make a good prediction and you'll get an error message.

If this happens, we recommend not setting the water heater fuel type when it's not required.

In [15]:
# Attempt to model a home with an uncommon heating fuel and water heater fuel combination.

response = requests.get(
    url=REM_ADDRESS_URL,
    headers=headers,
    # It's very unusual to use propane for home heating with natural gas for water heating.
    params=dict(address=address, upgrade=upgrade, heating_fuel="propane", water_heater_fuel="natural_gas"),
)

print(response)

# The error message will let you know if the water heater fuel is likely to be the problem.
response.text

<Response [400]>


'{"detail":{"type":"building_not_supported","msg":"The current version of this API cannot predict energy savings for the provided address. Omitting water_heater_fuel may help, by allowing more samples to match the characteristics of the provided home."}}'

In [56]:
# Omit the water heater fuel to get results.
response = requests.get(
    url=REM_ADDRESS_URL,
    headers=headers,
    params=dict(address=address, upgrade=upgrade, heating_fuel="propane"),
)

response

<Response [200]>

## Estimation Methodology
You may want to understand more about the *specificity* of the estimate returned. The `estimate_type` returned in a REM request indicates how much we understand about a home’s features before generating an estimate.

There are two potential outputs for `estimate_type`:
- `address_level`: The estimate is based on known features specific to the home at that address, enabling a more tailored result.
- `puma_level`: The estimate is based on the provided heating fuel and typical features of homes within the Census Public Use Microdata Area (PUMA) where the home is located.

The difference in precision between these two can be observed in the ratio between `percentile_80` and `percentile_20` where the `address-level` estimates will have a smaller ratio than the `puma-level` estimates.

### Address-Level Example

In [57]:

address_with_known_housing_features = "3126 Russell Street, San Diego, CA 92107" 

response = requests.get(
    url=REM_ADDRESS_URL,
    headers=headers,
    params=dict(address=address_with_known_housing_features, upgrade="hvac__heat_pump_seer18_hspf10", heating_fuel=heating_fuel),
)
data = response.json()
estimate_type = data["estimate_type"]

print(f"Performed address-level estimate due to full set of available housing features.\nEstimate type: {estimate_type}")

Performed address-level estimate due to full set of available housing features.
Estimate type: address_level


In [ ]:
percentile_80 = data["fuel_results"]["natural_gas"]["upgrade"]["emissions"]["percentile_80"]["value"]
percentile_20 = data["fuel_results"]["natural_gas"]["upgrade"]["emissions"]["percentile_20"]["value"]
percentile_ratio = percentile_80 / percentile_20

# A smaller ratio means estimates are more precise.
print(f"80th percentile: {percentile_80:.2f} kgCO2e\n20th percentile: {percentile_20:.2f} kgCO2e")
print(f"Observed ratio: {percentile_ratio:.2f}")


80th percentile: 2104.93 kgCO2e
20th percentile: 823.39 kgCO2e
Observed ratio: 2.56


### PUMA-level Example

In [59]:
address_with_limited_known_housing_features = "1382 US-67 Stephenville, TX 76401"

response = requests.get(
    url=REM_ADDRESS_URL,
    headers=headers,
    params=dict(address=address_with_limited_known_housing_features, upgrade="hvac__heat_pump_seer18_hspf10", heating_fuel=heating_fuel),
)
data = response.json()
estimate_type = data["estimate_type"]


print(f"PUMA-level estimate performed due to limited housing feature data.\nEstimate type: {estimate_type}")

PUMA-level estimate performed due to limited housing feature data.
Estimate type: puma_level


In [ ]:
percentile_80 = data["fuel_results"]["natural_gas"]["upgrade"]["emissions"]["percentile_80"]["value"]
percentile_20 = data["fuel_results"]["natural_gas"]["upgrade"]["emissions"]["percentile_20"]["value"]
percentile_ratio = percentile_80 / percentile_20 # In kgCO2e

# Larger ratio means estimates have more uncertaintity or variability
print(f"80th percentile: {percentile_80:.2f} kgCO2e\n20th percentile: {percentile_20:.2f} kgCO2e")
print(f"Observed ratio: {percentile_ratio:.2f}")

80th percentile: 1304.73 kgCO2e
20th percentile: 357.85 kgCO2e
Observed ratio: 3.65
